In [1]:
import torch
import torchvision
from torch import nn
from torch import optim
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.utils import save_image
from torchvision.datasets import MNIST
import os


In [2]:
if not os.path.exists('./vae_img'):
    os.mkdir('./vae_img')


def to_img(x):
    x = x.clamp(0, 1)
    x = x.view(x.size(0), 1, 28, 28)
    return x


In [3]:
class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()

        self.fc1 = nn.Linear(784, 400)
        self.fc21 = nn.Linear(400, 20)
        self.fc22 = nn.Linear(400, 20)
        self.fc3 = nn.Linear(20, 400)
        self.fc4 = nn.Linear(400, 784)

    def encode(self, x):
        h1 = F.relu(self.fc1(x))
        return self.fc21(h1), self.fc22(h1)

    def reparametrize(self, mu, logvar):
        std = logvar.mul(0.5).exp_()
        if torch.cuda.is_available():
            eps = torch.cuda.FloatTensor(std.size()).normal_()
        else:
            eps = torch.FloatTensor(std.size()).normal_()
        eps = Variable(eps)
        return eps.mul(std).add_(mu)

    def decode(self, z):
        h3 = F.relu(self.fc3(z))
        return F.sigmoid(self.fc4(h3))

    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparametrize(mu, logvar)
        return self.decode(z), mu, logvar


In [4]:
def loss_function(recon_x, x, mu, logvar):
    """
    recon_x: generating images
    x: origin images
    mu: latent mean
    logvar: latent log variance
    """
    BCE = reconstruction_function(recon_x, x)  # mse loss
    # loss = 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    KLD_element = mu.pow(2).add_(logvar.exp()).mul_(-1).add_(1).add_(logvar)
    KLD = torch.sum(KLD_element).mul_(-0.5)
    # KL divergence
    return BCE + KLD

In [5]:
num_epochs = 100
batch_size = 128
learning_rate = 1e-3

img_transform = transforms.Compose([
    transforms.ToTensor()
    # transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

dataset = MNIST('./data', transform=img_transform, download=True)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

model = VAE()
if torch.cuda.is_available():
    model.cuda()

reconstruction_function = nn.MSELoss(size_average=False)

optimizer = optim.Adam(model.parameters(), lr=1e-3)

for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    for batch_idx, data in enumerate(dataloader):
        img, _ = data
        img = img.view(img.size(0), -1)
        img = Variable(img)
        if torch.cuda.is_available():
            img = img.cuda()
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(img)
        loss = loss_function(recon_batch, img, mu, logvar)
        loss.backward()
        train_loss += loss.data[0]
        optimizer.step()
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch,
                batch_idx * len(img),
                len(dataloader.dataset), 100. * batch_idx / len(dataloader),
                loss.data[0] / len(img)))

    print('====> Epoch: {} Average loss: {:.4f}'.format(
        epoch, train_loss / len(dataloader.dataset)))
    if epoch % 10 == 0:
        save = to_img(recon_batch.cpu().data)
        save_image(save, './vae_img/image_{}.png'.format(epoch))

torch.save(model.state_dict(), './vae.pth')

Processing...
Done!
Train Epoch: 0 [0/60000 (0%)]	Loss: 183.709747


/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:34: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:41: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


Train Epoch: 0 [12800/60000 (21%)]	Loss: 51.337727
Train Epoch: 0 [25600/60000 (43%)]	Loss: 43.495949
Train Epoch: 0 [38400/60000 (64%)]	Loss: 39.463840
Train Epoch: 0 [51200/60000 (85%)]	Loss: 38.365051
====> Epoch: 0 Average loss: 45.5803
Train Epoch: 1 [0/60000 (0%)]	Loss: 36.135548
Train Epoch: 1 [12800/60000 (21%)]	Loss: 34.876072
Train Epoch: 1 [25600/60000 (43%)]	Loss: 33.696060
Train Epoch: 1 [38400/60000 (64%)]	Loss: 33.892876
Train Epoch: 1 [51200/60000 (85%)]	Loss: 33.307556
====> Epoch: 1 Average loss: 35.1271
Train Epoch: 2 [0/60000 (0%)]	Loss: 33.927097
Train Epoch: 2 [12800/60000 (21%)]	Loss: 33.718094
Train Epoch: 2 [25600/60000 (43%)]	Loss: 33.017609
Train Epoch: 2 [38400/60000 (64%)]	Loss: 33.364990
Train Epoch: 2 [51200/60000 (85%)]	Loss: 32.442837
====> Epoch: 2 Average loss: 33.2538
Train Epoch: 3 [0/60000 (0%)]	Loss: 31.611500
Train Epoch: 3 [12800/60000 (21%)]	Loss: 32.095543
Train Epoch: 3 [25600/60000 (43%)]	Loss: 31.691576
Train Epoch: 3 [38400/60000 (64%)]	Lo

Train Epoch: 28 [25600/60000 (43%)]	Loss: 30.105228
Train Epoch: 28 [38400/60000 (64%)]	Loss: 28.444664
Train Epoch: 28 [51200/60000 (85%)]	Loss: 28.852636
====> Epoch: 28 Average loss: 29.3043
Train Epoch: 29 [0/60000 (0%)]	Loss: 29.288792
Train Epoch: 29 [12800/60000 (21%)]	Loss: 29.583843
Train Epoch: 29 [25600/60000 (43%)]	Loss: 30.936615
Train Epoch: 29 [38400/60000 (64%)]	Loss: 29.512413
Train Epoch: 29 [51200/60000 (85%)]	Loss: 29.605236
====> Epoch: 29 Average loss: 29.2677
Train Epoch: 30 [0/60000 (0%)]	Loss: 28.870108
Train Epoch: 30 [12800/60000 (21%)]	Loss: 28.584343
Train Epoch: 30 [25600/60000 (43%)]	Loss: 28.965866
Train Epoch: 30 [38400/60000 (64%)]	Loss: 28.085285
Train Epoch: 30 [51200/60000 (85%)]	Loss: 29.018261
====> Epoch: 30 Average loss: 29.2246
Train Epoch: 31 [0/60000 (0%)]	Loss: 28.308573
Train Epoch: 31 [12800/60000 (21%)]	Loss: 28.426659
Train Epoch: 31 [25600/60000 (43%)]	Loss: 28.671312
Train Epoch: 31 [38400/60000 (64%)]	Loss: 28.769703
Train Epoch: 31 [

Train Epoch: 56 [25600/60000 (43%)]	Loss: 26.886427
Train Epoch: 56 [38400/60000 (64%)]	Loss: 28.099598
Train Epoch: 56 [51200/60000 (85%)]	Loss: 29.323936
====> Epoch: 56 Average loss: 28.6220
Train Epoch: 57 [0/60000 (0%)]	Loss: 28.900547
Train Epoch: 57 [12800/60000 (21%)]	Loss: 27.997540
Train Epoch: 57 [25600/60000 (43%)]	Loss: 27.172966
Train Epoch: 57 [38400/60000 (64%)]	Loss: 28.231377
Train Epoch: 57 [51200/60000 (85%)]	Loss: 28.572067
====> Epoch: 57 Average loss: 28.6236
Train Epoch: 58 [0/60000 (0%)]	Loss: 28.787979
Train Epoch: 58 [12800/60000 (21%)]	Loss: 28.486635
Train Epoch: 58 [25600/60000 (43%)]	Loss: 28.211384
Train Epoch: 58 [38400/60000 (64%)]	Loss: 28.844704
Train Epoch: 58 [51200/60000 (85%)]	Loss: 29.157852
====> Epoch: 58 Average loss: 28.6225
Train Epoch: 59 [0/60000 (0%)]	Loss: 29.493900
Train Epoch: 59 [12800/60000 (21%)]	Loss: 28.969484
Train Epoch: 59 [25600/60000 (43%)]	Loss: 29.101681
Train Epoch: 59 [38400/60000 (64%)]	Loss: 29.139744
Train Epoch: 59 [

Train Epoch: 84 [25600/60000 (43%)]	Loss: 27.612846
Train Epoch: 84 [38400/60000 (64%)]	Loss: 27.958817
Train Epoch: 84 [51200/60000 (85%)]	Loss: 28.648518
====> Epoch: 84 Average loss: 28.3492
Train Epoch: 85 [0/60000 (0%)]	Loss: 29.203257
Train Epoch: 85 [12800/60000 (21%)]	Loss: 28.752777
Train Epoch: 85 [25600/60000 (43%)]	Loss: 29.391205
Train Epoch: 85 [38400/60000 (64%)]	Loss: 28.371208
Train Epoch: 85 [51200/60000 (85%)]	Loss: 27.343235
====> Epoch: 85 Average loss: 28.3183
Train Epoch: 86 [0/60000 (0%)]	Loss: 28.250309
Train Epoch: 86 [12800/60000 (21%)]	Loss: 28.119259
Train Epoch: 86 [25600/60000 (43%)]	Loss: 28.935797
Train Epoch: 86 [38400/60000 (64%)]	Loss: 28.092178
Train Epoch: 86 [51200/60000 (85%)]	Loss: 28.010101
====> Epoch: 86 Average loss: 28.3190
Train Epoch: 87 [0/60000 (0%)]	Loss: 29.455128
Train Epoch: 87 [12800/60000 (21%)]	Loss: 28.264370
Train Epoch: 87 [25600/60000 (43%)]	Loss: 28.342337
Train Epoch: 87 [38400/60000 (64%)]	Loss: 27.640537
Train Epoch: 87 [